In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

fashion-mnist_test.csv
fashion-mnist_train.csv
t10k-images-idx3-ubyte
t10k-labels-idx1-ubyte
train-images-idx3-ubyte
train-labels-idx1-ubyte



In [2]:
train=pd.read_csv("../input/fashion-mnist_train.csv")
test=pd.read_csv("../input/fashion-mnist_test.csv")

In [3]:
train.shape

(60000, 785)

In [4]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


In [14]:
X=np.array(train.iloc[:,1:])
y=to_categorical(np.array(train.iloc[:,0]))

In [15]:
X_train,X_val,y_train,y_valt=train_test_split(X,y,test_size=0.1,random_state=42)

In [16]:
test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_test=np.array(test.iloc[:,1:])
y_test=to_categorical(np.array(test.iloc[:,0]))

In [19]:
X_train=X_train.reshape(X_train.shape[0],28,28,1)
X_val=X_val.reshape(X_val.shape[0],28,28,1)
X_test=X_test.reshape(X_test.shape[0],28,28,1)

In [27]:
X_train=X_train/255.0
X_val=X_val/255.0
X_test=X_test/255.0

In [30]:
X_train.dtype

dtype('float64')

In [31]:
X_train=X_train.astype('float32')
X_val=X_val.astype('float32')
X_test=X_test.astype('float32')


In [32]:
X_train.dtype

dtype('float32')

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [34]:
batch_size=256
num_epochs=50
num_classes=10
input_shape=(28,28,1)

In [38]:
def build_model():
    model =Sequential()
    model.add(Conv2D(filters=32,kernel_size=3,strides=(1,1),input_shape=input_shape,activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64,kernel_size=3,strides=(1,1),activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=128,kernel_size=3,strides=(1,1),activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    

In [39]:
model=build_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         73856     
__________

In [41]:
history=model.fit(X_train,y_train,
                  epochs=num_epochs,
                  batch_size=batch_size,
                  verbose=1,
                  validation_data=(X_val, y_valt))
scores=model.evaluate(X_test, y_test, verbose=0)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 60s 1ms/step - loss: 0.8969 - acc: 0.6598 - val_loss: 0.5751 - val_acc: 0.7835
Epoch 2/50
54000/54000 [==============================] - 61s 1ms/step - loss: 0.5823 - acc: 0.7841 - val_loss: 0.4789 - val_acc: 0.8263
Epoch 3/50
54000/54000 [==============================] - 62s 1ms/step - loss: 0.5129 - acc: 0.8107 - val_loss: 0.4161 - val_acc: 0.8425
Epoch 4/50
54000/54000 [==============================] - 62s 1ms/step - loss: 0.4664 - acc: 0.8292 - val_loss: 0.3795 - val_acc: 0.8600
Epoch 5/50
54000/54000 [==============================] - 62s 1ms/step - loss: 0.4326 - acc: 0.8405 - val_loss: 0.3616 - val_acc: 0.8675
Epoch 6/50
54000/54000 [==============================] - 62s 1ms/step - loss: 0.4101 - acc: 0.8489 - val_loss: 0.3347 - val_acc: 0.8773
Epoch 7/50
54000/54000 [==============================] - 62s 1ms/step - loss: 0.3922 - acc: 0.8557 - val_loss: 0.3188 - val_acc:

In [42]:
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.235116134453
Test accuracy: 0.916
